In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_cell();
        return false;
    }}
);


<IPython.core.display.Javascript object>

In [2]:
# 연산자의 중복과 장식자
# 연산자의 중복(Operator overloading) 이란 프로그램 언어에서 지원하는 연산자에 대해 클래스가 새로운 동작을 하도록 정의
# 파이썬에서 사용하는 모든 연산자는 클래스 내에서 새롭게 정의할 수 있다.

# 이항 연산자.
# 파이썬에서는 내장 자로형에 사용하는 모든 연산을 독자가 정의하는 클래스에서 동작하도록 구현할 수 있다.

class MyStr:
    def __init__(self, s):
        self.s = s
        # 여기에 현재는 가능하지 않은 나눗셈 연산 추가해보자. 나누기(/) 연산자는 __truediv__() 메서드로 확장.
        # 예를들면 s1 / ':'에 의해 s1.__truediv__(':')가 호출
    def __truediv__(self, b):
        return self.s.split(b)

s1 = MyStr('a:b:c')
s1 / ':' # 나누기 연산이 가능해 짐

['a', 'b', 'c']

In [3]:
# 이처럼 파이썬은 모든 연산자에 대응하는 적절한 이름의 메서드가 정해져 있어서 연산자가 사용될 때 해당 메서드로 확장된다.
# 더하기 연산자 추가

class MyStr:
    def __init__(self, s):
        self.s = s
    def __truediv__(self, b):
        return self.s.split(b)
    def __add__(self, b):
        return self.s + b

    
s1 = MyStr('a:b:c')

s1.s + ':d' # s1.__add__(':d')

'a:b:c:d'

In [4]:
# 이처럼 파이썬은 모든 연산자에 대응하는 적절한 이름의 메서드가 정해져 있어서 연산자가 사용될 때 해당 메서드로 확장된다.
# 더하기 연산자 안 쓰면 아래와 같이 s1.s 처럼 일일이 접근해서 써야함.
class MyStr:
    def __init__(self, s):
        self.s = s
    def __truediv__(self, b):
        return self.s.split(b)r
#     def __add__(self, b):
#         return self.s + b
    
s1 = MyStr('a:b:c')
s1.s + ':d' # s1.__add__(':d')

SyntaxError: invalid syntax (<ipython-input-4-be1d27c0ccc1>, line 7)

In [6]:
# 역 이항 연산자
# 만약 'z:' + s1 한다면?? 'z:'이 MyStr형 객체와의 연산을 지원하지 않는다.
# a+b 연산에서 a.__add__(b) 를 우선 시도하고, 이것이 구현되어 있지 않으면 b.__radd__(a) 시도한다.

class MyStr:
    def __init__(self, s):
        self.s = s
    def __truediv__(self, b):
        return self.s.split(b)
    def __add__(self, b):
        return self.s + b
    def __radd__(self, b):
        return b + self.s # __add__ 와 순서 반대로임.  r 붙어있음.
    
s1 = MyStr('a:b:c')
'z:'+s1


'z:a:b:c'

In [25]:
# 확장 산술 연산자 
class MyStr:
    def __init__(self, s):
        self.s = s
    def __truediv__(self, b):
        return self.s.split(b)
    def __add__(self, b):
        return self.s + b
    def __radd__(self, b):
        return b + self.s # __add__ 와 순서 반대로임.  r 붙어있음.
    def __iadd__(self, b):
        self.s = self.s + b
        return self.s
    def get(self):
        return self.s 
    
s1 = MyStr('a:b:c')
print('z:'+s1)
print(s1.__iadd__('sdf'))
print(s1.s)

# 단항 연산자, 형변환 연산자 모두 연산자 중복 가능.

z:a:b:c
a:b:csdf
a:b:csdf


In [32]:
# 다음은 __index() 메서드를 사용하는 간단한 예
class Index:
    def __index__(self):
        print('__index__called')
        return 3
L = [1, 2, 3, 4, 5]
i = Index()
L[i:]
bin(i)


# 인덱스로 사용할 자료형은 정수로만 표현해야 하는데, 이를 검사하기 위해서 __index__() 메서드가 존재

__index__called
__index__called


'0b11'

In [15]:
# 컨테이너 자료형(시퀀스 자료형, 매핑 자료형)의 연산자 중복
# 이 절에서는 컨테이너 자료형에 적용할 메서드 소개.

# 인덱싱
# 인덱싱은 시퀀스 자료형엥서 순서에 의해서 데이터에 접근하기 위한 방법을 제공. 기본적으로는 __getitem__()메서드 정의
# 다음 Square 클래스는 생성자 인수로 범위를 받아들이며, 해당 범위에서 요구된 인덱스 값의 제곱을 반환하는 시퀀스 자료형.
# 숫자 리스트를 주기 보단, length 를 줘서 그 범위 안에서 숫자가 뽑히도록 만들어 놓은 것.
# 시퀀스 자료형에서의 인덱스는 정수 값으로 전달하며, 만일 인덱스 범위를 초과하면 indexError 에러를 발생해야 한다.
# 만약 데이터의 자료형이 맞지 않으면 TypeError 에러를 발생한다.

class Square:
    def __init__(self, end):
        self.end = end
    def __len__(self):
        return self.end
    def __getitem__(self, k):
        if type(k) != int: 
            raise TypeError('...')
        if k < 0 or self.end <= k:
            raise IndexError('index {} out of range'.format(k))
        return k * k
    
s1 = Square(10) # s1 은 길이가 10인 시퀀스형 데이터 클래스. 그 숫자들은 __getitem__ 에 있는데, 0~9 까지이다.(범위로 줬음)
# 그러니까 시퀀스형 데이터를 클래스로 만들 때, 일일이 다 만들 필요 없이 저렇게 범위로 만들면 된다!
len(s1)
print(s1[4])
# print(s1['a'])#이건 error 가 나옴

for x in s1:
    print(x, end = ' ') 
# for 문은 인스턴스 객체 s2의 __getitem()__메서드를 0부터 호출하기 시작한다.
# 이 메서드에 의해서 반환된 값이 x에 전달되고, 내부의 print() 함수가 실행된다.
# IndexError 예외가 발생하면 반복을 중단한다. 인스턴스 객체 s1은 제한된 범위 내에서 시퀀스형 객체로서의 역할을 충실히 수행
# 시퀀스 자료형인 클래스를 만들기 위해서는, 시퀀스 자료형에 대한 메서드 목록을 참고해서 정의하면 된다.

16


TypeError: ...

In [18]:
print(list(s1)) # 자료형 변환. __getitems__() 메서드만 정의되어 있으면, 다른 시퀀스 자료형으로 변환하는 것이 가능
print(tuple(s1))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
(0, 1, 4, 9, 16, 25, 36, 49, 64, 81)


In [7]:
# 슬라이싱
# 그니까, 내가 새로운 클래스를 만들었는데 이 클래스가 시퀀스 형이길 원할 때. 이 시퀀스로 슬라이싱 하기를 원할 때
# 이 클래스로 사전 자료형 처리를 하고 싶을 때. 이 클래스로 연산을 하고 싶을 때!! 클래스에 연산자 중복을 하는 거. 
# 클래스를 설계하는 거임. 이 클래스가 어떤 연산을 하게끔 하는가 미리 설계하고, 그에 맞는 메서드 넣어주고!

# 슬라이싱에서는 인덱싱에서와 같이 _getitem__()와 __setitem__(), __delitem__() 메서드를 사용하지만 
# 인수로 정수가 아닌 slice 객체를 전달.
# 우선 slice 객체를 살펴보자. 이 객체는 start와 step, stop 세 개의 멤버를 가지는 단순간 객체. 
# 형식은 slice([start,] stop [, step])

s = slice(1, 10, 2)
s
print(type(s))
print(s.start, s.stop, s.step)

# 만약에 slice 객체를 전달할 때 인수가 생략되면 None 객체를 기본 값으로 가진다
print(slice(10))
print(slice(1,10))
print(slice(1, 10, 3))

# 슬라이싱 m[1:5]는 m.__getitem__(slice(1,5))를 호출한다. 즉, 인덱싱의 정수 인덱스 대신에 slice 객체가 범위를 나타내는데 사용.
# 확장 슬라이싱 m[1:10:2] 는 m._getitem__(slice(1, 10, 2))를 호출한다.

class Square:
    def __init__(self, end):
        self.end = end
    def __len__(self):
        return self.end
    def __getitem__(self, k):
        if type(k) == slice:
            start = k.start or 0
            stop = k.stop or self.end
            step = k.step or 1
            return map(self.__getitem__, range(start, stop, step))
        elif type(k) == int:
            if k < 0 or self.end <= k:
                raise IndexError(k)
            return k * k
        else:
            raise TypeError('...')
s = Square(10)
print(s[4]) # 인덱싱
print(list(s[1:5])) # 슬라이싱
print(list(s[1:10:2])) # 간격은 2로
print(s[:]) # 전체 범위

# __getitem__() 메서드의 정의를 보면, 가장 먼저 k의 자료형 검사. k가 slice형이면 슬라이싱을, 아니면 인덱싱을 적용한다.
# 슬라이싱 부분에서 start와 stop, step을 별도의 지역 변수에 치환한 이유는 range()함수가 정수 인수만을 요구하기 때문.
# 최종적으로 map()함수에 의해서 각 인덱스 값의 제곱에 대한 리스트 반환.

<class 'slice'>
1 10 2
slice(None, 10, None)
slice(1, 10, None)
slice(1, 10, 3)
16
[1, 4, 9, 16]
[1, 9, 25, 49, 81]


In [12]:
# 매핑 자료형
# 매핑 자료형에서 object.__getitem__(self, key) 등의 메서드의 key 는 사전의 키로 사용할 수 있는 임의의 객체가 될 수 있다.
# 만일 key에 대응하는 값을 찾을 수 없으면 KeyError 에러를 발생. 다음 클래스 MyDict 는 치환(d[key]=value), 크기(len(d))정보를
# 얻을 수 있는 간단한 클래스

class MyDict:
    def __init__(self):
        self.d = {}
    def __getitem__(self, k): # key
        return self.d[k]
    def __setitem__(self, k, v):
        self.d[k] = v
    def __len__(self):
        return len(self.d)

m = MyDict() # __init__() k\메서드 호출
m['day'] = 'light' # m.__setitem__('day', 'light')
m.__setitem__('night', 'darkness')
print(m['day']) # m.__getitem__('day')
print(m['night']) # m.__getitem__('night')
print(len(m))

# 여태까지 int, float, list, tuple, str, dictionary 이런 자료형에 대해서 +,-,*,/, append, sort 이런 메서드를 
# 사용했던 건, 저 자료형에 이런 메서드들이 정의되어 있었기 때문.
# 지금도 클래스를 만들면 클래스는 하나의 새로운 자료형이기 때문에, 그 자료형이 사용 할 수 있는 연산자들을 우리가 정할 수 있는데
# 우리가 보통 사용하는 연산자 그대로 쓰기 위해서 위와 같이 지정된 name 을 가지고 메서드를 만드는 것임. 
# 뭐 어려운 게 아니고, 내가 어떤 클래스(자료형)을 만드냐에 따라, 시퀀싱을 하느냐, 걍 사칙연산만 하느냐 아니면 둘다 하느냐
# 또 키를 가지고 매핑하느냐 설계하면됨!!

light
darkness
2


In [ ]:
# 문자열 변환 연산
# 인스턴스 객체를 print() 함수로 출력할 때 내가 원하는 형식으로 출력하거나, 인스턴스 객체를 사람이 읽기 좋은 형태로
# 변환하려면 문자열로 변환하는 기능이 필요하다. 이 절에서는 인스턴스 객체를 문자열로 변환하는 기능에 대해서 살펴보자
# 문자열로의 변환 : __str__()와 __repr__() 메서드 
# 인스턴스 객체를 문자열로 변환하는 메서드는 __str__()와 __repr__() 두가지이다. 이 두메서드는 호출되는 시점이 조금 다름.

class StringRepr:
    def __repr__(self):
        return 'repr called'
    def __str__(self):
        return 'str called'
s = StringRepr()
print(s)
str(s)
repr(s)

# print() 함수와 str() 함수에 의해서 __str__() 메서드가 호출. repr() 함수에 의해서 __repr__() 메서드가 호출. 
# __repr__() 메서드의 목적은 객체를 대표해서 유일하게 표현할 수 있는 문자열을 만들어 내는 것.
# 즉 다른 객체의 출력과 혼동되지 않는 모양으로 표현해야 한다는 의미

In [22]:
repr(2)

'2'

In [24]:
repr('2')

"'2'"

In [25]:
repr('abc') # 문자열 'abc'에 대한 repr 문자열 표현

"'abc'"

In [26]:
repr([1, 2, 3]) # fltmxm [1, 2, 3]에 대한 repr 문자열 표현

'[1, 2, 3]'

In [27]:
# 다음으로 __str__() 메서드의 목적은 사용자가 읽기 편한 형태의 표현으로 출력
str(2)

'2'

In [28]:
str('2') # repr('2') 는 "2'"

'2'

In [29]:
# 재밌는 건 컨테이너 자료형(리스트와 사전 등)의 __str__()메서드는 내부 객체의 __repr__() 메서드 사용
L = [2, '2']
str(L)

"[2, '2']"

In [ ]:
repr(L)

In [31]:
str(L) == repr(L)

True

In [3]:
# 만일 __str__() 메서드를 호출할 상황에서 __str__()메서드가 정의되어 있지 않으면 __repr__() 메서드가 대신 호출
class StringRepr:
    def __repr__(self):
        return 'repr called'

s = StringRepr()

print(str(s))
print(repr(s))
print(s)

repr called
repr called
repr called


In [4]:
# 그러나 __repr_() 메서드가 정의되어 있지 않은 경우에 __str__() 메서드가 __repr__() 메서드를 대신하지는 않는다.
# __repr__() 은 특별함, 대표성을 가짐. 자신은 남을 대체할 수 있으나, 다른 이가 자신을 대체할 수는 없다.

class StringRepr:
    def __str__(self):
        return 'str called'
    
s = StringRepr()
print(str(s))
print(repr(s))

str called


In [6]:
# 바이트로의 변환 : __bytes__() 메서드
# 문자열이 아닌 바이트 자료형으로 변환하려면 __bytes__() 메서드를 사용. b.__bytes__() 메서드는 bytes(b) 함수에 의해서 호출

class BytesRepr:
    def __bytes__(self):
        return 'bytes called'.encode('utf-8')

b = BytesRepr()
bytes(b)

b'bytes called'

In [17]:
# 서식 기호 새로 지정하기: __format__() 메서드
# __format__() 메서드는 format() 함수나 문자열의 format() 메서드에 의해서 호출된다. 
# format(x, "o") # x.__format__("o") 호출
# "x:{:o}".format(x) # x.__format__("o") 호출

# __format__() 메서드에 전달되는 변환 기호는 사용자가 새로 정의 할 수 있음.
# 다음은 대문자 변환을 위한 새로운 변환 기호 u와 소문자 변환을 위한 새로운 변환 기호 l을 정의하는 예
# 변환 기호가 요구될 때는 __format__() 메서드가 호출

class MyStr:
    def __init__(self, s):
        self.s = s
    def __format__(self, fmt):
        print(fmt) # 서식 문자열 확인
        if fmt[0] == 'u': # u이면 대문자로 변환
            s = self.s.upper()
            fmt = fmt[1:]
        elif fmt[0] == 'l': # l이면 소문자로 변환
            s = self.s.lower()
            fmt = fmt[1:]
        else:
            s = str(self.s)
        return s.__format__(fmt)
    
# 순서가 채우기 문자가 u, l 아니면 다른거인지 확인하고
# 반환할 string 만들고.
# 그리고 채우기 문자 뒤에는 format 형식이니, 기존의 __format__ 함수 이용하여 format에 맞게 출력.
    
s = MyStr('Hello')
print('{0:u^20} {0:l} {0:*^20}'.format(s))  # * : 채우기 문자, ^ : 가운데 정렬, 20:확보자리수

u^20
l
*^20
here
       HELLO         hello *******Hello********


In [18]:
# 진릿값과 비교 연산
# __bool__() 메서드
# 클래스 인스턴스의 진릿값은 __bool__() 메서드의 반환 값으로 결정됨.
# 만일 이 메서드가 정의되어 있지않으면 __len__() 메서드를 호출한 결과가 0이면 False로 간주하고, 아니면 true로 간주.
# 만일, __len__()과 __bool__()메서드 모두가 정의되어 있지 않으면 모든 인스턴스는 True가 된다.

class Truth:
    def __init__(self, num):
        self.num = num
    def __bool__(self):
        return self.num != 0
    
print(bool(Truth(0)))
print(bool(Truth(3)))

False
True


In [24]:
# 비교 연산
# 파이썬의 모든 비교 연산은 중복이 간으하도록 메서드 이름이 준비되어 있다.
# x<y 는 x.__lt__(y) 메서드로 확장되며, x<=y는 x.__le__(y) 메서드로 확장. 다른 연산자도 같은 방식
# __eq__()와 __ne__()메서드는 각자의 논리로 적용될 수 있다. 즉, o==other이 참이라고해서 o!=other이 거짓이 아닐 수도 있다는 것.
# 하지만 __ne__() 메서드가 정의되어 있지 않고, __eq__()메서드만 정의되어 있으면 o!=other 은 not(o==other) 의 논리 적용

class Compare:
    def __init__(self, n):
        self.n = n
    def __eq__(self, o):
        print('__eq__ called')
        return self.n == o
    def __lt__(self, o):
        print('__lt__ called')
        return self.n < o
    def __le__(self, o):
        print('__le__ called')
        return self.n <= o

c = Compare(10)
print(c<10) # c라는 객체 자체에 < operator 를 적용한 건데, c가 가지고 있는 특정한 값과 10을 비교하여 true, false 결과 줌.
print(c<=10)
# c>10 는 정의되어 있지 않아서 error 나옴
print(c==10)
print(c!=10) # __ne__()가 정의 되어 있으면 __ne__(), 아니면 not __eq__() 

__lt__ called
False
__le__ called
True
__eq__ called
True
__eq__ called
False


In [27]:
# 해시 값에 접근하기 : __hash__()메서드
# 해시 값을 돌려주는 내장 함수 hash(m)가 호출될 때, m.__hash__() 메서드가 호출된다.
# hash()함수를 사용한 예로, 사전은 (키, 값)쌍을 저장할 때 키에 대한 hash()함수의 호출 결과를 값을 저장하기 위한 해시 키로 사용.
# __hash__() 메서드는 정수를 반환해야 함. 
# 이 메서드를 정의한 클래스는 __eq__() 메서드도 함께 정의해야 해시가 가능한 객체로 취급된다.

class Obj:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def _key(self):
        return (self.a, self.b)
    def __eq__(self, o):
        return self._key() == o._key()
    def __hash__(self):
        return hash(self._key())
    
o1 = Obj(1, 2)
o2 = Obj(3, 4)
print(hash(o1))
print(hash(o2))
d = {o1:1, o2:2}
print(d)

# 해시 키는 변경이 가능해서는 안 된다. 만일 변경 간으한 자료형으로 클래스를 정의하면 hash()함수를 호출했을 때 
# TypeError 에러를 반환해서 해시 키로 사용할 수 없도록 해야 한다.

class Obj2:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __hash__(self):
        raise TypeError('not proper type')
        
o1 = Obj2(1, 2)
d = {o1:1}

1299869600
1699342716
{<__main__.Obj object at 0x00BB8E30>: 1, <__main__.Obj object at 0x00BB8E90>: 2}


TypeError: not proper type